<a href="https://colab.research.google.com/github/ekacs/scrap-inaportnet-22/blob/main2/SPB_webscrap_INAPORTNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import re
from six.moves import urllib
from unicodedata import normalize
import numpy as np
import io
import datetime
from datetime import datetime, timedelta
from dateutil.tz import tzutc, tzlocal
from google.colab import files


In [13]:
kode_lokasi='IDSUB'     #diisi kode lokasi pelabuhan, cari tahu di https://bit.ly/KODE_PELABUHAN 
tahun=2023              #diisi tahun
jenis_layanan='SPB'    #diisi kode layanan inaportnet yg mau discrape, SPB
berapa_bulan=5       #diisi berapa bulan
selisih_waktu_UTC=+7     # diisi waktu selisih perbedaan dg waktu UTC, WIB=+7, WITA=+8, WITT=+9

#all code in here

In [14]:

daftarbulan=['00','01','02','03','04','05','06','07','08','09','10','11','12']
urls=[]
daftarlink=[]   


for i in range(1,berapa_bulan+1):
  
  #FYI gaes syntax format URL = https://monitoring-inaportnet.dephub.go.id/site/port_detail_service?unlocode=<kode_lokasi>&bulan=<angka_bulan_2digit>&tahun=<angka_tahun_4digit>&service_code=<kode_layanan>
  urlpage = ('https://monitoring-inaportnet.dephub.go.id/site/port_detail_service?unlocode={kd_lokasi}&bulan={n}&tahun=2022&service_code={jns_produk}').format(kd_lokasi=kode_lokasi, n=(daftarbulan[i]), jns_produk=jenis_layanan)
  urls.append(urlpage)


for url in urls:
    my_url = requests.get(url) 
  
    soup = bs(my_url.content, 'html.parser')
    #print(soup)
    s = soup('a', class_='btn btn-primary ls-modal')
    #print(s)
    for link in soup.find_all('a', attrs={'href': re.compile("kode_perusahaan=")}):
      daftarlink.append('https://monitoring-inaportnet.dephub.go.id'+link.get('href')+'&download=1')
      #daftarlink.append(link.get('href'))
      #print(daftarlink)

#len(daftarlink)
#print(urls)
#print(urlpage)


hasil_scrape = pd.DataFrame(pd.read_csv(daftarlink[0]))

for i in range(0,len(daftarlink)):
  data = pd.read_html(daftarlink[i])
  df = pd.DataFrame(data[0])

  hasil_scrape = pd.concat([hasil_scrape,df],axis=0)

#hasil_scrape
#hasil_scrape.dropna(subset=["No"])

#PERHITUNGAN SELISIH WAKTU

df=hasil_scrape

df['Waktu Permohonan'] = pd.to_datetime(
    df['Waktu Permohonan'],
    format='%d-%m-%Y %H:%M:%S'
)
df['Waktu Respon'] = pd.to_datetime(
    df['Waktu Respon'],
    format='%d-%m-%Y %H:%M:%S'
)

print(df.dtypes)


df['waktu_permohonan_real'] = df['Waktu Permohonan']+pd.DateOffset(hours=selisih_waktu_UTC) 
df['selisih_waktu'] = df['Waktu Respon']-df['waktu_permohonan_real']
#hasil_scrape.info()



#cari data anomali Waktu Respon lebih dulu ketimbang Waktu Permohonan (asumsi ada bug di database inaportnet)

hasil_scrape_bug=hasil_scrape[hasil_scrape['selisih_waktu']<timedelta(seconds=0)]
hasil_scrape_ok=hasil_scrape[hasil_scrape['selisih_waktu']>timedelta(seconds=0)]



#make criteria in time difference
def sel(hasil_scrape):
    if hasil_scrape['selisih_waktu'] <= timedelta(seconds=30):
        return "Antara 0 detik - 30 detik"
    elif hasil_scrape['selisih_waktu'] <= timedelta(minutes=30):
        return "Antara 31 detik - 30 menit"
    elif hasil_scrape['selisih_waktu'] <= timedelta(hours=4):
        return "Antara 31 menit - 4 jam"
#    elif hasil_scrape['selisih_waktu'] < timedelta(hours=1):
#        return "Antara 30 - 60 menit"
#    elif hasil_scrape['selisih_waktu'] <= timedelta(hours=6):
#        return "Antara 1 - 6 jam"
#    elif hasil_scrape['selisih_waktu'] < timedelta(hours=12):
#        return "Antara 6 - 12 jam"
#    elif hasil_scrape['selisih_waktu'] < timedelta(days=1):
#        return "Antara 12 - 24 jam"
    else:
        return "Lebih dari 4 jam"
#        return "Lebih dari 24 jam"


hasil_scrape_ok['lama_pelayanan'] = hasil_scrape_ok.apply(sel, axis=1)
sort_data = hasil_scrape_ok.sort_values("selisih_waktu", axis = 0, ascending = True)



lama_pelayanan = hasil_scrape_ok.groupby('lama_pelayanan')['lama_pelayanan'].count().reset_index(name='jumlah')
lama_pelayanan['persentase'] = lama_pelayanan['jumlah']/len(hasil_scrape_ok)*100
#lama_pelayanan


#gruoping month-day pelayanan

month_mapping={
    1: 'January', 
    2: 'February', 
    3: 'March', 
    4: 'April', 
    5: 'May',
    6: 'June', 
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
} 
dw_mapping={
    0: 'Monday', 
    1: 'Tuesday', 
    2: 'Wednesday', 
    3: 'Thursday', 
    4: 'Friday',
    5: 'Saturday', 
    6: 'Sunday'
} 


hasil_scrape_ok['bulan permohonan'] = hasil_scrape_ok['waktu_permohonan_real'].dt.month.map(month_mapping)
hasil_scrape_ok['hari permohonan'] = hasil_scrape_ok['waktu_permohonan_real'].dt.dayofweek.map(dw_mapping)

bulan_permohonan = hasil_scrape_ok.groupby('bulan permohonan')['bulan permohonan'].count().reset_index(name='jumlah')
bulan_permohonan['persentase'] = bulan_permohonan['jumlah']/len(hasil_scrape_ok)*100

def nomor(bulan_permohonan):
    if bulan_permohonan['bulan permohonan'] == 'January':
        return 1
    elif bulan_permohonan['bulan permohonan'] == 'February':
        return 2
    elif bulan_permohonan['bulan permohonan'] == 'March':
        return 3
    elif bulan_permohonan['bulan permohonan'] == 'April':
        return 4
    elif bulan_permohonan['bulan permohonan'] == 'May':
        return 5
    elif bulan_permohonan['bulan permohonan'] == 'June':
        return 6
    elif bulan_permohonan['bulan permohonan'] == 'July':
        return 7
    elif bulan_permohonan['bulan permohonan'] == 'August':
        return 8
    elif bulan_permohonan['bulan permohonan'] == 'September':
        return 9
    elif bulan_permohonan['bulan permohonan'] == 'October':
        return 10
    elif bulan_permohonan['bulan permohonan'] == 'November':
        return 11
    else:
        return 12

bulan_permohonan['nomor'] = bulan_permohonan.apply(nomor, axis=1)
bulan_permohonan = bulan_permohonan.sort_values("nomor", axis = 0, ascending = True)
#bulan_permohonan

#stat hari permohnonan

def nomor_hari(hari_permohonan):
    if hari_permohonan['hari permohonan'] == 'Monday':
        return 1
    elif hari_permohonan['hari permohonan'] == 'Tuesday':
        return 2
    elif hari_permohonan['hari permohonan'] == 'Wednesday':
        return 3
    elif hari_permohonan['hari permohonan'] == 'Thursday':
        return 4
    elif hari_permohonan['hari permohonan'] == 'Saturday':
        return 5
    elif hari_permohonan['hari permohonan'] == 'Sunday':
        return 6
    else:
        return 7
    
hari_permohonan = hasil_scrape_ok.groupby('hari permohonan')['hari permohonan'].count().reset_index(name='jumlah')
hari_permohonan['Nomor'] = hari_permohonan.apply(nomor_hari, axis=1)
hari_permohonan = hari_permohonan.sort_values("Nomor", axis = 0, ascending = True)
#hari_permohonan


#pemabgian rentang waktu permohonan
batas_waktu1 = pd.to_datetime('06:00:00', format='%H:%M:%S')
batas_waktu2 = pd.to_datetime('12:00:00', format='%H:%M:%S')
batas_waktu3 = pd.to_datetime('18:00:00', format='%H:%M:%S')

def jam(hasil_scrape_ok):
    if hasil_scrape_ok['jam permohonan'] < batas_waktu1:
        return "Jam 00.00 s.d. 06.00"
    elif hasil_scrape_ok['jam permohonan'] < batas_waktu2:
        return "Jam 06.00 s.d. 12.00"
    elif hasil_scrape_ok['jam permohonan'] < batas_waktu3:
        return "Jam 12.00 s.d. 18.00"
    else:
        return "Jam 18.00 s.d. 24.00"

hasil_scrape_ok['jam permohonan'] = hasil_scrape_ok['waktu_permohonan_real'].dt.time

hasil_scrape_ok['jam permohonan'] = pd.to_datetime(hasil_scrape_ok['jam permohonan'], format='%H:%M:%S')
hasil_scrape_ok['pembagian jam permohonan'] = hasil_scrape_ok.apply(jam, axis=1)

waktu_permohonan = hasil_scrape_ok.groupby('pembagian jam permohonan')['pembagian jam permohonan'].count().reset_index(name='jumlah')
waktu_permohonan['persentase'] = waktu_permohonan['jumlah']/len(hasil_scrape_ok)*100
#waktu_permohonan


<table class="table table-bordered">            object
No                                             float64
Nama Kapal                                      object
Waktu Permohonan                        datetime64[ns]
Waktu Respon                            datetime64[ns]
Nomor Produk                                    object
dtype: object


<ipython-input-14-52ef61fbd650>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hasil_scrape_ok['lama_pelayanan'] = hasil_scrape_ok.apply(sel, axis=1)
<ipython-input-14-52ef61fbd650>:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hasil_scrape_ok['bulan permohonan'] = hasil_scrape_ok['waktu_permohonan_real'].dt.month.map(month_mapping)
<ipython-input-14-52ef61fbd650>:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

#data yg anomali (asumsi ada bug di dBasenya, karena Waktu Respon mendahului Waktu Permohonan)

In [16]:
'''
hasil_scrape_bug.dropna(subset=['No']).to_csv('hasil_scrape_bug.csv')
files.download('hasil_scrape_bug.csv')
'''
hasil_scrape_bug.dropna(subset=['No'])

#JUMLAH RECORD DATANYA
print(len(hasil_scrape_bug.index))


26


#statistik durasi lama pelayanan

In [17]:
lama_pelayanan

,lama_pelayanan,jumlah,persentase
0,Antara 31 detik - 30 menit,345,6.696429
1,Antara 31 menit - 4 jam,3112,60.403727
2,Lebih dari 4 jam,1695,32.899845


#statistik waktu pelayanan

In [18]:
waktu_permohonan

,pembagian jam permohonan,jumlah,persentase
0,Jam 00.00 s.d. 06.00,333,6.463509
1,Jam 06.00 s.d. 12.00,1160,22.515528
2,Jam 12.00 s.d. 18.00,1978,38.392857
3,Jam 18.00 s.d. 24.00,1681,32.628106


#statistik hari pelayanan

In [19]:
hari_permohonan

,hari permohonan,jumlah,Nomor
1,Monday,721,1
5,Tuesday,687,2
6,Wednesday,751,3
4,Thursday,752,4
2,Saturday,824,5
3,Sunday,647,6
0,Friday,770,7


#statistik bulan pelayanan

In [20]:
bulan_permohonan

,bulan permohonan,jumlah,persentase,nomor
2,January,1067,20.710404,1
1,February,970,18.827640,2
4,March,1074,20.846273,3
0,April,1107,21.486801,4
5,May,927,17.993012,5
3,June,7,0.135870,6


#unduh hasil scrape (aslinya)

In [21]:
#hasil scrape asli blm ada pengolahan data (termasuk didalamnya ada bug)
hasil_scrape.dropna(subset=['No']).to_csv('hasil_scrape_asli.csv')
files.download('hasil_scrape_asli.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#unduh hasil scrape (+ kolom tambahan)

In [22]:
#hasil scrape asli sudah ada kolom tambahannya dan TIDAK termasuk didalamnya ada asumsi data bug

#hasil_scrape_ok.head()
hasil_scrape_ok.dropna(subset=['No']).to_csv('hasil_scrape_ok.csv')
files.download('hasil_scrape_ok.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#ngintip data hasil scrape asli sudah ada kolom tambahannya dan TIDAK termasuk didalamnya ada asumsi data bug
hasil_scrape_ok